In [9]:
# cleaning format of elements in csv files - getting rid of b' at beginning and ' at end 

def clean_csv(infile, outfile):
    inputfile = open(infile, 'r')
    outputfile = open(outfile, 'w')
    for row in inputfile:
        new_row = row.replace("b'", '')
        new_row_2 = new_row.replace("'", '')
        #print(new_row_2)
        outputfile.write(new_row_2+'\n')
    return outputfile

clean_csv("nodes1.csv", "nodes.csv")    
clean_csv("nodes_tags1.csv", "nodes_tags.csv")
clean_csv("ways1.csv", "ways.csv")
clean_csv("ways_nodes1.csv", "ways_nodes.csv")
clean_csv("ways_tags1.csv", "ways_tags.csv")

<_io.TextIOWrapper name='ways_tags.csv' mode='w' encoding='cp1252'>

In [10]:
'''  Creating tables using sqlite, database name is Minneapolis.db
     Note the following key relationships:
        Nodes has id as primary key
        Ways has id as primary key
        Nodes_Tags has id as foreign key referencing the Nodes table primary key
        Ways_Tags has id as foreign key referencing the Ways table primary key
        Ways_Nodes has id as foreign key referencing the Ways table primary key
            and node_id as foreign key referencing the Nodes table primary key
'''
import import_ipynb
import data
import sqlite3
import pandas as pd
from pandas import DataFrame
dbm = db = sqlite3.connect(':memory:') # data base stored in local memory
sqlite3.connect('Minneapolis.db')
c = db.cursor()

# Create Nodes table
c.execute("CREATE TABLE Nodes ([id] integer primary key not null, [lat] float, [lon] float, [user] text, [uid] integer, [version] text, [changeset] integer, [timestamp] text);")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\nodes.csv')
read_clients.to_sql('Nodes', db, if_exists='append', index = False)
db.commit()

# Create Nodes_Tags table
c.execute("CREATE TABLE Nodes_Tags ([id] integer, [key] text, [value] text, [type] text, foreign key (id) references nodes(id));")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\nodes_tags.csv')
read_clients.to_sql('Nodes_Tags', db, if_exists='append', index = False)
db.commit()

# Create Ways table
c.execute("CREATE TABLE Ways ([id] integer primary key not null, [user] text, [uid] integer, [version] text, [changeset] integer, [timestamp] text);")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\ways.csv')
read_clients.to_sql('Ways', db, if_exists='append', index = False)
db.commit()

# Create Ways_Tags table
c.execute("CREATE TABLE Ways_Tags ([id] integer, [key] text, [value] text, [type] text, foreign key (id) references ways(id));")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\ways_tags.csv')
read_clients.to_sql('Ways_Tags', db, if_exists='append', index = False)
db.commit()

# Create Ways_Nodes table
c.execute("CREATE TABLE Ways_Nodes ([id] integer, [node_id] integer, [position] integer, foreign key (id) references ways(id), foreign key (node_id) references nodes(id));")
read_clients = pd.read_csv(r'C:\Users\maxgs\DataWrangling_MongoDB\project\ways_nodes.csv')
read_clients.to_sql('Ways_Nodes', db, if_exists='append', index = False)
db.commit()

In [11]:
# number of node tags
c.execute('''select count(*) from Nodes''')
print(c.fetchall())

[(269731,)]


In [12]:
# number of way tags
c.execute('''select count(*) from Ways''')
print(c.fetchall())

[(45657,)]


In [13]:
# number of unique users
c.execute('''select count(distinct(u.uid))
             from (select uid from nodes union all select uid from ways) u;''')
print(c.fetchall())

[(627,)]


In [14]:
# top 10 frequently contributing users
c.execute('''select u.user, count(*) as num
            from (select user from Nodes union all select user from Ways) u
            group by u.user
            order by num desc
            limit 10;''')
print(c.fetchall())

[('Mulad', 78844), ('iandees', 66917), ('sota767', 15770), ('Arun Balaji', 13191), ('DavidF', 12344), ('houston_mapper1', 11305), ('neuhausr', 11008), ('nickrosencrans', 8711), ('GOKUL606', 8681), ('samhandler', 6997)]


In [15]:
# number of users with only one contribution
c.execute('''select count(*) from
             (select u.user, count(*) as num
              from (select user from nodes union all select user from ways) u
              group by u.user
              having num = 1) use;''')
print(c.fetchall())

[(128,)]


In [17]:
# top 5 amenities
c.execute('''select value, count(*) as num
             from nodes_tags
             where key = "amenity"
             group by value
             order by num desc
             limit 5;''')
print(c.fetchall())

[('restaurant', 291), ('bicycle_parking', 203), ('bench', 136), ('cafe', 100), ('bicycle_rental', 97)]
